In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime
from statsmodels.regression.rolling import RollingOLS
# import wrds

# DATA

In [2]:
his_members = pd.read_excel("../data/members.xlsx", sheet_name=1)
his_members.head()

,1980-03-31,1980-06-30,1980-09-30,1980-12-31,1981-03-31,1981-06-30,1981-09-30,1981-12-31,1982-03-31,1982-06-30,...,2020-09-30,2020-12-31,2021-03-31,2021-06-30,2021-09-30,2021-12-31,2022-03-31,2022-06-30,2022-09-30,2022-10-20
0,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,...,1981718D UW Equity,2061517D UW Equity,2061517D UW Equity,2150994D UW Equity,2150994D UW Equity,1950546D US Equity,2150994D UW Equity,2150994D UW Equity,A UN Equity,A UN Equity
1,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,...,2061517D UW Equity,2078185D UN Equity,2150994D UW Equity,9990213D UN Equity,9990213D UN Equity,2150994D UW Equity,9990213D UN Equity,9990213D UN Equity,AA UN Equity,AA UN Equity
2,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,...,2078185D UN Equity,2150994D UW Equity,9990213D UN Equity,9990253D UN Equity,9990253D UN Equity,9990213D UN Equity,9990253D UN Equity,A UN Equity,AAL UW Equity,AAL UW Equity
3,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,...,2150994D UW Equity,9990213D UN Equity,9990253D UN Equity,9990527D UW Equity,9990527D UW Equity,9990253D UN Equity,9990527D UW Equity,AA UN Equity,AAP UN Equity,AAP UN Equity
4,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,...,9990213D UN Equity,9990253D UN Equity,9990620D UN Equity,A UN Equity,A UN Equity,9990527D UW Equity,A UN Equity,AAL UW Equity,ACGL UW Equity,ACGL UW Equity


In [3]:
ave_ecoc = pd.read_excel("../clean_data/ave_ecoc_1117.xlsx", index_col=0)
FCFROIC_ADJ = pd.read_excel(
    "../clean_data/FCFROIC_ADJ_yearly_0112.xlsx", index_col=0)
ave_ecoc.head()

,beta,name,DATES,GT10 Govt,rf,coe,10y_avg
183,1.138810,A,1994-06-30,7.319975,0.073200,0.141528,0.180326
184,1.133677,A,1994-07-29,7.110731,0.071107,0.139128,0.179694
185,1.110756,A,1994-08-31,7.173438,0.071734,0.138380,0.179047
186,1.106904,A,1994-09-30,7.603406,0.076034,0.142448,0.178495
187,1.110309,A,1994-10-31,7.806800,0.078068,0.144687,0.177961


In [6]:
ave_ecoc.index = ave_ecoc["DATES"]

In [7]:
FCFROIC_ADJ.head()

,Year,Ticker Symbol,NYear,Cash & Short-Term Investments,Total Debt,Total Shareholder Equity,Total Invested Capital (Calculated),FCF,SIC,ROE,avg IC,FCFROIC,FCFROIC_ADJ,avg_fcfroic,avg_fcfroic_expanding
0,1999,A,-23,789.0,0.0,4171.0,3382.0,27.0,3826.0,NaN,3202.0,0.008432,0.008432,NaN,NaN
1,2000,A,-22,996.0,110.0,5265.0,4379.0,14.0,3826.0,NaN,3880.5,0.003608,0.003608,NaN,NaN
2,2001,A,-21,1170.0,6.0,5659.0,4495.0,621.0,3826.0,NaN,4437.0,0.139959,0.139959,NaN,NaN
3,2002,A,-20,1844.0,1150.0,4627.0,3933.0,-799.0,3826.0,NaN,4214.0,-0.189606,-0.189606,NaN,NaN
4,2003,A,-19,1607.0,1150.0,2824.0,2367.0,-349.0,3826.0,NaN,3150.0,-0.110794,-0.110794,NaN,NaN


In [9]:
close_price = pd.read_excel("../data/price_data_0112.xlsx")
close_price.head()

,Ticker,Monthly Calendar Date,Monthly Price,Monthly Price Flag,Monthly Price Date,Factor To Adjust Price
0,ACF,1970-01-30,46.375,TR,1970-01-30,NaN
1,ACF,1970-02-27,48.125,TR,1970-02-27,0.0
2,ACF,1970-03-31,49.250,TR,1970-03-31,NaN
3,ACF,1970-04-30,45.500,TR,1970-04-30,NaN
4,ACF,1970-05-29,38.500,TR,1970-05-29,0.0


In [7]:
db = wrds.Connection(wrds_username='zixin0370')

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
Loading library list...
Done


In [8]:
db.create_pgpass_file()

In [9]:
db.close()
db = wrds.Connection(wrds_username='zixin0370')

Loading library list...
Done


In [18]:
"crsp_a_stock62"in sorted(db.list_libraries())

True

In [13]:
data = db.get_table(library='djones', table='djdaily', columns=['date', 'dji'], obs=10)
data

,date,dji
0,1896-05-26,40.94
1,1896-05-27,40.58
2,1896-05-28,40.20
3,1896-05-29,40.63
4,1896-06-01,40.60
5,1896-06-02,40.04
6,1896-06-03,39.77
7,1896-06-04,39.94
8,1896-06-05,40.32
9,1896-06-08,39.81


In [31]:
db = wrds.Connection(wrds_username='zixin0370')
params = {"tickers": ("AAL")}
tikers = ("AAL")
crsp_m = db.raw_sql("""
                      select a.date, permco, permno, ret, shrout 
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date < '11/30/1966' and b.ticker IN %(tickers)s
                      """%(tikers), params=params)
crsp_m 

There was an error with your password.


OperationalError: (psycopg2.OperationalError) connection to server at "wrds-pgdata.wharton.upenn.edu" (165.123.60.118), port 9737 failed: FATAL:  too many connections for role "zixin0370"

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [24]:
db.close()